환경설정

In [3]:
import os
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import tqdm
import jpype
import ast

In [4]:
folder_path = '/Users/jaesolshin/Documents/GitHub/youtube_dashboard'
file_path = os.path.join(folder_path, 'KPOP_comments_merged_preprocessed_with_nouns.csv')
#file_path = os.path.join(folder_path, 'KPOP_comments_merged_preprocessed.csv')
comments_df = pd.read_csv(file_path)
print(len(comments_df))

648060


In [5]:
comments_df.head()

,Group,Title,comment_type,comment_id_key,parent_id_key,comment,author,date,likes,korean,word_list
0,NMIXX,별별별 (See that),node,UgyiK-QaarUUumBCjUJ4AaABAg,NaN,엔믹스 노래 중독성 있고 좋은데 왜 안뜨는지 진심 의문..,@lumi1216,2024-10-01T13:40:14Z,4,True,"['중독', '진심', '노래', '의문', '엔믹스']"
1,NMIXX,별별별 (See that),node,Ugw_6c4z7iSqIjzfgZB4AaABAg,NaN,대쉬랑 비슷한것 같기도하고 이젠 좀 다른 대중성있는 노래로갔으면하는 아쉬움이있네요 ...,@써니노,2024-10-01T09:56:20Z,1,True,"['기도', '아쉬움', '대쉬', '금도', '대중성', '노래']"
2,NMIXX,별별별 (See that),node,UgxLdDuk-Ji3RKARe_h4AaABAg,NaN,4본부장아 피씨로 엘범 클릭하면 별별별 뮤비로 연결 안되고 수정별인가 그 장면 있는...,@고해진-i4g,2024-10-01T05:16:15Z,4,True,"['간다', '본부장', '엘범', '천만', '한번', '클릭', '연락', '연..."
3,NMIXX,별별별 (See that),node,Ugz-Tnp-d-MLAQSXvQF4AaABAg,NaN,이야 노래 종나 멋있네,@박종한-i9u,2024-09-30T18:33:22Z,10,True,['노래']
4,NMIXX,별별별 (See that),node,UgwlfRfRmVnAsHIREut4AaABAg,NaN,외모췍을 필두로 더더더더 빵빵뜨길...리더로서 해원이 얼마나 노력했을까...멤버들 ...,@egi3581,2024-09-30T17:22:58Z,6,True,"['걸그룹', '멤버', '빵빵', '노력', '묵자', '얼마나', '세대', '..."


# 키워드 추출

In [6]:
import time
from collections import defaultdict

def extract_keywords_frequency(series_word_list, top_n=30, min_df=1, existing_keywords=None):
    # 시작시간 확인
    start_time = time.time()

    # 기존 키워드가 없으면 빈 리스트로 초기화
    if existing_keywords is None:
        existing_keywords = []

    # 단어별 문서 빈도를 저장할 딕셔너리 (각 단어가 등장한 문서 수)
    doc_frequency = defaultdict(int)

    # 각 문서에서 등장한 단어 추출 및 빈도 계산
    for word_list in series_word_list:
        unique_words_in_doc = list(set(word_list))  # 문서 내에서 중복된 단어는 한 번만 카운트
        for word in unique_words_in_doc:
            doc_frequency[word] += 1

    # min_df 조건을 충족하는 단어들의 빈도를 저장할 딕셔너리
    word_frequencies = defaultdict(int)

    # 각 문서의 단어 빈도 카운트 (min_df 조건에 맞는 단어만 빈도 계산)
    for word_list in series_word_list:
        for word in word_list:
            if doc_frequency[word] >= min_df:
                word_frequencies[word] += 1

    # 상위 빈도 단어 추출
    sorted_words = sorted(word_frequencies.items(), key=lambda item: item[1], reverse=True)

    # 기존 키워드 제외하고 새로운 키워드만 추출
    new_keywords_with_weights = [(word, count) for word, count in sorted_words if word not in existing_keywords]
    
    # 상위 n개 키워드만 추출 (중복 제외 후)
    top_keywords_with_weights = new_keywords_with_weights[:top_n]

    # 키워드만 추출
    top_keywords = [word for word, _ in top_keywords_with_weights]

    # 가중치를 전체 합으로 표준화
    total_count = sum(count for _, count in top_keywords_with_weights)
    if total_count > 0:
        keywords_with_weights = [(word, count / total_count) for word, count in top_keywords_with_weights]
    else:
        keywords_with_weights = [(word, 0) for word, count in top_keywords_with_weights]

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights


In [7]:
# TF-IDF 기반 키워드 및 상대적 비중 추출 함수
def extract_keywords_tfidf(series_word_list, top_n=100, min_df=1):

    # 시작시간 확인
    start_time = time.time()

    # 리스트 시리즈를 문자열 시리즈로 변형
    word_list_str = series_word_list.apply(lambda words: ' '.join(words))

    # TF-IDF 벡터화 (min_df를 설정해 빈도수가 적은 단어 제거)
    vectorizer = TfidfVectorizer(max_features=1000, min_df=min_df)
    tfidf_matrix = vectorizer.fit_transform(word_list_str)

    # 각 단어의 TF-IDF 점수를 추출
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = np.mean(tfidf_matrix.toarray(), axis=0)  # 전체 문서에서 평균 TF-IDF 추출

    # TF-IDF 점수를 기준으로 상위 키워드 추출
    top_n_indices = tfidf_scores.argsort()[-top_n:][::-1]
    top_keywords = [feature_names[idx] for idx in top_n_indices]

    # 각 단어의 TF-IDF 점수 추출
    weights = [tfidf_scores[idx] for idx in top_n_indices]

    # 가중치를 전체 합으로 표준화
    weights = weights / np.sum(weights)

    # 키워드와 가중치를 묶어서 반환
    keywords_with_weights = list(zip(top_keywords, weights))

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights

In [8]:
from rank_bm25 import BM25Okapi
import time
import numpy as np
import pandas as pd
from collections import defaultdict

# BM25 기반 키워드 및 상대적 비중 추출 함수 (min_df 추가)
def extract_keywords_bm25(series_word_list, top_n=100, min_df=1):
    # 시작시간 확인
    start_time = time.time()

    # 시리즈의 각 리스트를 문서로 간주하여 BM25 모델 생성
    tokenized_corpus = series_word_list.tolist()
    bm25 = BM25Okapi(tokenized_corpus)

    # 각 단어의 출현 빈도를 저장할 딕셔너리
    doc_frequency = defaultdict(int)

    # 문서 내 각 단어의 출현 빈도 계산
    for doc in tokenized_corpus:
        unique_words_in_doc = set(doc)
        for word in unique_words_in_doc:
            doc_frequency[word] += 1

    # 각 단어의 BM25 점수를 저장할 딕셔너리
    word_scores = defaultdict(float)
    total_docs = len(tokenized_corpus)

    # 각 문서에서의 BM25 점수를 계산하여 해당 단어에 대한 점수를 합산
    for doc in tokenized_corpus:
        scores = bm25.get_scores(doc)
        for word, score in zip(doc, scores):
            # 단어의 출현 빈도가 min_df 이상인 경우에만 점수 계산
            if doc_frequency[word] >= min_df:
                word_scores[word] += score

    # 문서 전체에 대한 평균 BM25 점수로 변환
    word_scores = {word: score / total_docs for word, score in word_scores.items()}

    # 상위 BM25 점수를 기준으로 키워드 추출
    sorted_words = sorted(word_scores.items(), key=lambda item: item[1], reverse=True)
    top_keywords = sorted_words[:top_n]

    # 가중치를 전체 합으로 표준화
    total_weight = sum(score for _, score in top_keywords)
    keywords_with_weights = [(word, score / total_weight) for word, score in top_keywords]

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights

In [9]:
sample_df = comments_df.copy()
#sample_df = comments_df.sample(frac=0.1, random_state=42)
print(len(sample_df), "\n")

# csv 파일로 저장되면서 문자열로 변형된 word_list 컬럼을 다시 리스트로 변환
sample_df['word_list'] = sample_df['word_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 단순 빈도 기반 키워드 추출
print('**단순 빈도 기반 키워드 추출**\n')
keywords1, keywords_with_weights1 = extract_keywords_frequency(sample_df['word_list'], top_n=300, min_df=50)

# 결과 출력
for item in keywords_with_weights1: print(item)
print('\n')

# TF-IDF 기반 키워드 추출
print('**TF-IDF 기반 키워드 추출**\n')
keywords2, keywords_with_weights2 = extract_keywords_tfidf(sample_df['word_list'], top_n=300, min_df=50)

# 결과 출력
for item in keywords_with_weights2: print(item)
print('\n')

648060 

**단순 빈도 기반 키워드 추출**

648060개 댓글 키워드 추출에 걸린 시간: 0.6802222728729248 초

('노래', 0.06271202408694847)
('뉴진스', 0.04163983720910315)
('뮤비', 0.027531500443415992)
('아이브', 0.021116567407406254)
('하이브', 0.020515483743879185)
('사랑', 0.020461759944937)
('민희진', 0.018448285393277673)
('단월드', 0.014559149513332457)
('컨셉', 0.01017404175039923)
('멤버', 0.010047907613752358)
('응원', 0.009203120340531024)
('에스파', 0.009018590770251342)
('사이비', 0.008679118649399267)
('화이팅', 0.008637073937183644)
('한국', 0.008595029224968019)
('가사', 0.00777437873098158)
('데뷔', 0.007539239784886795)
('아이돌', 0.007203660692943574)
('삭제', 0.007089984248805034)
('하나', 0.007009787853282641)
('중독', 0.006845502033328999)
('르세라핌', 0.006663308280394628)
('대표', 0.006644621741632129)
('여기', 0.006157214522243599)
('방시혁', 0.006017844087306623)
('음악', 0.00600538639479829)
('앨범', 0.0058426577864081905)
('이제', 0.005621533744385279)
('리즈', 0.005598954176713925)
('파트', 0.005553795041371218)
('아이', 0.005511750329155594)
('컴백', 0.005343571

In [10]:
# 출력 옵션 설정: 모든 열을 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.max_rows', None)     # 모든 행 출력

# 데이터프레임 전치 및 출력
compare_result  = pd.DataFrame([keywords1, keywords2]).transpose()
compare_result

,0,1
0,노래,노래
1,뉴진스,뉴진스
2,뮤비,아이브
3,아이브,사랑
4,하이브,뮤비
5,사랑,하이브
6,민희진,민희진
7,단월드,단월드
8,컨셉,화이팅
9,멤버,에스파


# 추가적인 불용어, 교정이 필요한 단어 찾아내기

In [11]:
## keyword_list 초기화 및 설정
keyword_list = pd.DataFrame(columns=['Group', 'Title', 'keyword'])

## 모든 댓글에 대한 키워드 추가
#sample_df['word_list'] = sample_df['word_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
# 단순 빈도 기반 키워드 추출
keywords, _ = extract_keywords_frequency(sample_df['word_list'], top_n=300, min_df=50)
keyword_list['keyword'] = pd.DataFrame(keywords)
keyword_list['Group'] = 'All'
keyword_list['Title'] = 'All'

## 각 그룹에 대한 추가키워드 발견
# Group 열에 속한 그룹들의 리스트
groups = sample_df['Group'].unique()

# 기존 keyword_list에서 추출된 키워드와 중복되지 않도록 새로운 키워드 추출
existing_keywords = keyword_list['keyword'].tolist()

# 그룹별로 새로운 키워드 추출
for group in groups:
    word_list = sample_df[sample_df['Group'] == group]['word_list']
    new_keywords, _ = extract_keywords_frequency(word_list, top_n=30, min_df=50, existing_keywords=existing_keywords)
    
    # 새로운 키워드를 기존 리스트에 추가
    frac = pd.DataFrame()
    frac['keyword'] = new_keywords
    frac['Group'] = group
    frac['Title'] = 'All'
    
    keyword_list = pd.concat([keyword_list, frac])
    keyword_list.drop_duplicates(subset='keyword', keep='first')
    existing_keywords = keyword_list['keyword'].tolist()
## 각 뮤비에 대한 추가 키워드 발견
# Title 열에 속한 그룹들의 리스트
titles = sample_df['Title'].unique()

# 그룹별로 새로운 키워드 추출
for title in titles:
    group = sample_df[sample_df['Title'] == title]['Group'].iloc[0] # 해당 타이틀에 속한 그룹 조회
    word_list = sample_df[sample_df['Title'] == title]['word_list'] # 댓글 시리즈 가져오기
    new_keywords, _ = extract_keywords_frequency(word_list, top_n=10, min_df=50, existing_keywords=existing_keywords)
    
    # 새로운 키워드를 기존 리스트에 추가
    frac = pd.DataFrame()
    frac['keyword'] = new_keywords
    frac['Group'] = group
    frac['Title'] = title
    
    keyword_list = pd.concat([keyword_list, frac])
    keyword_list.drop_duplicates(subset='keyword', keep='first')
    existing_keywords = keyword_list['keyword'].tolist()

# 인덱스 초기화
keyword_list = keyword_list.reset_index(drop=True)

648060개 댓글 키워드 추출에 걸린 시간: 0.7101829051971436 초

23511개 댓글 키워드 추출에 걸린 시간: 0.017847061157226562 초

50699개 댓글 키워드 추출에 걸린 시간: 0.03480172157287598 초

249967개 댓글 키워드 추출에 걸린 시간: 0.3420381546020508 초

45376개 댓글 키워드 추출에 걸린 시간: 0.04448413848876953 초

7425개 댓글 키워드 추출에 걸린 시간: 0.0046961307525634766 초

136993개 댓글 키워드 추출에 걸린 시간: 0.10666584968566895 초

33181개 댓글 키워드 추출에 걸린 시간: 0.02482891082763672 초

7846개 댓글 키워드 추출에 걸린 시간: 0.006060123443603516 초

69125개 댓글 키워드 추출에 걸린 시간: 0.050428152084350586 초

23937개 댓글 키워드 추출에 걸린 시간: 0.016282081604003906 초

4339개 댓글 키워드 추출에 걸린 시간: 0.003738880157470703 초

2622개 댓글 키워드 추출에 걸린 시간: 0.0018308162689208984 초

551개 댓글 키워드 추출에 걸린 시간: 0.0004677772521972656 초

1442개 댓글 키워드 추출에 걸린 시간: 0.0010421276092529297 초

1406개 댓글 키워드 추출에 걸린 시간: 0.0008342266082763672 초

1416개 댓글 키워드 추출에 걸린 시간: 0.000865936279296875 초

1916개 댓글 키워드 추출에 걸린 시간: 0.0011038780212402344 초

1608개 댓글 키워드 추출에 걸린 시간: 0.0009450912475585938 초

570개 댓글 키워드 추출에 걸린 시간: 0.0003540515899658203 초

3550개 댓글 키워드 추출에 걸린 시간: 0.0028

In [12]:
keyword_list

,Group,Title,keyword
0,All,All,노래
1,All,All,뉴진스
2,All,All,뮤비
3,All,All,아이브
4,All,All,하이브
5,All,All,사랑
6,All,All,민희진
7,All,All,단월드
8,All,All,컨셉
9,All,All,멤버


In [13]:
keyword_list.to_csv('keyword_list_before_cleaning.csv', encoding='utf-8-sig', index=False)

In [14]:
# 특정 단어가 포함된 댓글 조회하기

def keyword_search(hot_word, dataframe=sample_df, likes=False, n_comments=10, random_state=42):

    # 데이터 프레임에서 hot_word를 포함하고 있는 행 필터링
    filtered_df = dataframe[dataframe['comment'].str.contains(hot_word, na=False)]
    print(len(filtered_df))

    # 좋아요 기준 정렬
    if likes:
        filtered_df = filtered_df[['comment','word_list','likes']].sort_values(by='likes', ascending=False)
        result = filtered_df[:n_comments]
    
    # 랜덤으로 조회
    else :
        result = filtered_df[['comment','word_list','likes']].sample(n = n_comments, random_state=42)
    
    return result

In [70]:
keyword_search('둥이', n_comments=200, likes=False)

966


,comment,word_list,likes
61854,핍둥이 다 각자만의 보컬 매력 포인트가 있으니깐 꼭 각 멤버별 솔로 커버곡 들을수 ...,"[각자, 멤버, 매력, 포인트, 커버, 둥이, 처럼, 보컬, 솔로, 스노우맨]",17
183339,진짜 넘 예쁘구 완벽한 사랑둥이들이다 진짜,[사랑둥이],30
371531,2022년도 함께하자 엣둥이들!!!,"[하자, 둥이]",1
222469,핌둥이들 거기서 도망쳐 ㅜㅜ,"[둥이, 거기]",8
40277,얼굴이 다 똑같아... 다섯쌍둥이야??,"[얼굴, 쌍둥이]",1
73090,조회수 르세에 확 밀려서 걱정했다 이젼 안심이다 겸둥이,"[르세, 안심, 이젼, 둥이, 조회수, 걱정]",8
604983,상식적으로 ㅋㅋ 한국 kpop에서 왜 중국요소로 가지고 활동한다는 게 말이 된다고 ...,"[문화, 요소, 건가, 약탈, 나라, 거의, 중국, 아이브, 노래, 한국, 탐욕, ...",6
319942,​@@ABC123-if3eb보니즈들 ㅠㅠ 방저씨가 우리 뉴둥이들 인사 안 받아줬대요...,"[둥이, 대요, 기분, 인사]",0
45099,오늘 아침도 상쾌하게 핍둥이들 노래로 시작~,"[시작, 둥이, 노래, 아침]",8
232202,미안해 핌둥이들.. 이 할미가 돈이 없어서 앨범을 못사..,"[둥이, 할미, 앨범]",2


In [66]:
group = 'STAYC'
hot_word = '치아'
sample_df[(sample_df['Group'] == group) & (sample_df['comment'].str.contains(hot_word, na=False))][['comment','word_list','likes']][:10]

,comment,word_list,likes
624891,스테이씨 인생은 언제나 LIT 하구만 치아땡이랑 비슷한 느낌인데 개인적으로 너무 좋...,"[인생, 치아, 스테이씨, 개인]",2
627790,치아땡 듣다가 못참고 소밷 들으러 다시 왔습니다 테디베어 런투유 등 히트곡 많지만 ...,"[런투유, 테디베어, 노래, 소밷, 히트, 자주, 소통, 활동, 원픽, 치아, 스테...",5
627793,아쉬운게 옛날 계정으로 스테이시 초창기에 들었던 기억 써놨는데 날아갔네 암튼 장동 ...,"[장동, 중소, 하이, 갑자기, 시도, 멤버, 커리어, 데뷔, 옛날, 기억, 노래,...",1
627794,너무 아쉬운게 내가 예전 계정으로 스테이시 처음 들었던 추억 같은 거 댓글로 썼는데...,"[장동, 중소, 상황, 멤버, 데뷔, 옛날, 노래, 떡볶이, 비주, 예전, 스테이시...",0
627861,뮤직뱅크 에서 스테이씨 무대에 거의 4년전 데뷔곡 무대를 만들어 스페셜 방송 했어요...,"[뮤직뱅크, 스페셜, 런투유, 공연, 스윗, 코로나, 배드, 스테이씨, 거의, 데뷔...",9
630841,치아땡보다 임펙트가 강한듯,"[치아, 임펙트]",1
635672,치아땡 시은이 무대 페이스직캠으로 홀려서 입덕함... 치아땡 시은은 신이다 ㄹㅇ 표...,"[입덕, 페이스, 표정, 무대, 직캠, 치아, 홀려]",0
635856,아무리봐도 치아땡 곡 구리단말은 이해가 안감. 솔직히 사운드나 구성자체도 오히려 이...,"[유광, 전혀, 처럼, 눈빛, 안감, 몰입, 베이지, 멤버, 소재, 아이, 엉덩이,...",2
635942,치아땡 챌린지,"[치아, 린지]",2
636019,치아땡 1위 축하해,[치아],2


In [17]:
title = '蜚蜚 (FEIFEI)'
hot_word = '홍콩'
sample_df[(sample_df['Title'] == title) & (sample_df['comment'].str.contains(hot_word, na=False))][['comment','word_list','likes']][:10]

,comment,word_list,likes
6964,원곡은 린이 부른 해를 품은 달ost 이고 홍콩 가수가 이 노래를 리메이크 했음.,"[가수, 노래, 홍콩, 리메이크]",4
6970,광동어임 홍콩쪽,"[광동어, 홍콩]",5
6975,광동어 노래라 그런지.. 90년대 홍콩영화 OST 인가 했다..,"[광동어, 노래, 홍콩영화]",0
6978,저거 원곡이 이미 한국껀데? Feifei는 홍콩인이 리메이크한 노래예요,"[껀데, 한국, 리메이크, 홍콩인, 노래]",0
6980,커버해주셔서 감사해요!! 너무 잘들었어요!! p.s.홍콩노래인데 번체자막 쓰셨으면 ...,"[홍콩, 자막, 커버, 번체, 노래]",2
6989,홍콩사람으로서 . 엔믹스 정말 너무 잘하셨어요!!! 처음 들었을 때… 엔익스의 광둥...,"[홍콩, 익스, 커버, 감동, 뮤비, 노래, 광둥어, 엔믹스]",13
6997,한국에서 드라마 OST 모두 대박쳤지만 홍콩에서도 많이 사랑받았구나,"[드라마, 한국, 홍콩]",1
6999,ㅇㅏ니 원래 보컬 춤 외모 라이브 성격이 다 완벽한 엔믹스 광둥어 음원까지 내준거ㅠ...,"[가사, 광둥어, 하나, 멤버, 외모, 문제, 그거, 바로, 라이브, 감동, 보컬,...",13
7001,이 노래는 원래 홍콩 돌어갔어 여가수 故陳僖儀 의 노래 입니다. 음원 부터 뮤비 까...,"[홍콩, 가수, 감동, 뮤비, 노래, 음원, 부터, 원래]",3
7015,원곡이 홍콩 가수 노래임,"[가수, 노래, 홍콩]",0
